In [34]:
import os
import re
from pprint import pprint


def extract_info(file_path):
    # Use regex to match the first two digits and the title
    match = re.match(r"(\d{2})-([\w-]+)", file_path)

    # If a match is found, extract the digits and the title
    if match:
        digits = int(match.group(1))  # Convert digits to integer
        title = match.group(2)
        return digits, title
    else:
        return None, None


# 1. 각 파일을 읽고
def read_readme_from_directories():
    dir_pattern = r"^\d{2}-"

    result = {}
    for dir_name in os.listdir():
        if os.path.isdir(os.path.join(dir_name)) and re.match(dir_pattern, dir_name):
            readme_path = os.path.join(dir_name, "translations/ko/README.md")
            if os.path.exists(readme_path):
                with open(readme_path, "r", encoding="utf-8") as readme_file:
                    content = readme_file.read()
                digits, title = extract_info(dir_name)
                result[digits] = {"title": title, "sections": split_markdown(content)}
    return result


# 2. 헤딩 (#, ##, ### …) 별로 나눈 뒤
def split_markdown(content):
    pattern = r"^(#+\s)"
    sections = re.split(pattern, content, flags=re.MULTILINE)[1:]
    return [sections[i] + sections[i + 1] for i in range(0, len(sections), 2)]


readme_files = read_readme_from_directories()
pprint(readme_files)

{0: {'sections': ['# 코스 시작하기\n'
                  '\n'
                  '이 코스를 시작하시게 되어 정말 기쁩니다. 여러분이 Generative AI와 함께 무엇을 영감받아 '
                  '만들어낼지 기대됩니다!\n'
                  '\n'
                  '여러분들의 시간을 성공적으로 보내기 위해, 코스 환경 설정, 기술적 요구사항, 도움이 필요한 경우 어떻게 '
                  '도움을 받을 수 있는지 안내하는 페이지를 만들었습니다.\n'
                  '\n',
                  '## 코스 환경 설정하기\n\n이 과정을 시작하려면 다음 단계를 완료해야 합니다.\n\n\n',
                  '### 1. 레포지토리 포크하기\n'
                  '\n'
                  '[레포지토리를 '
                  '포크](https://github.com/microsoft/generative-ai-for-beginners/fork?WT.mc_id=academic-105485-koreyst) '
                  '하여 자신의 Github 계정에 업로드하여 코드를 변경하고 과제를 완료할 수 있습니다. 또한 [레포지토리에 '
                  '스타 (🌟)를 '
                  '추가](https://docs.github.com/en/get-started/exploring-projects-on-github/saving-repositories-with-stars?WT.mc_id=academic-105485-koreyst) '
                  '하여 쉽게 찾아볼 수 있도록 할 수 있습니다.\n'
                  '\n',
                  '### 2. C

In [35]:
%pip install python-dotenv openai

Note: you may need to restart the kernel to use updated packages.


In [36]:
def save_to_file(file_path, content):
    os.makedirs(os.path.dirname(file_path), exist_ok=True)

    with open(file_path, "w") as file:
        file.write(content)

In [38]:
import asyncio
from openai import OpenAI, AsyncOpenAI
from dotenv import load_dotenv

load_dotenv()
client = AsyncOpenAI()


# 3. GPT-4o 를 사용하여 번역
async def translate_text(text):
    completion = await client.chat.completions.create(
        model="gpt-4o",
        messages=[
            {
                "role": "system",
                "content": """
                You are an AI expert. 
                Your job is to find any unatural expressions or incorrectness in the following text and correct them. 
                Only include the final markdown text in your response.
                """,
            },
            {"role": "user", "content": text},
        ],
    )
    print(completion)
    return completion.choices[0].message.content


async def transalte(file, i):
    tasks = [translate_text(section) for section in file["sections"]]
    translated_sections = await asyncio.gather(*tasks)
    content = "\n\n".join(translated_sections)
    save_to_file(f"{i:02d}-{file['title']}/translations/ko/README.md", content)


tasks = [transalte(file, i) for i, file in list(readme_files.items())]
await asyncio.gather(*tasks)

ChatCompletion(id='chatcmpl-ACHFMX9E1IGgEQpQFdgBVjbCmvGLM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='## Power Platform의 생성형 AI\n\n생성형 AI를 사용하여 low-code 개발과 애플리케이션을 향상시키는 것은 Power Platform의 주요 관심사입니다. 목표는 데이터 과학 전문 지식이 필요하지 않고 AI를 사용하여 앱, 사이트, 대시보드를 구축하고 프로세스를 자동화할 수 있도록 모든 사람에게 가능성을 제공하는 것입니다. 이 목표는 Copilot과 AI Builder의 형태로 생성형 AI를 Power Platform의 low-code 개발 환경에 통합함으로써 달성됩니다.', refusal=None, role='assistant', function_call=None, tool_calls=None))], created=1727489596, model='gpt-4o-2024-05-13', object='chat.completion', service_tier=None, system_fingerprint='fp_c17d3befe7', usage=CompletionUsage(completion_tokens=113, prompt_tokens=165, total_tokens=278, completion_tokens_details=CompletionTokensDetails(reasoning_tokens=0)))
ChatCompletion(id='chatcmpl-ACHFMdTatYNP3nItDepzKJfltdkxM', choices=[Choice(finish_reason='stop', index=0, logprobs=None, message=ChatCompletionMessage(content='## 잘하셨습니다. 학습을 계속하세요!\n\n이 레슨을 완료한 후 [Gener

[None]